In [ ]:
# 这行代码将你的Google Drive挂载到Colab虚拟机上
from google.colab import drive
drive.mount('/content/drive')

# 待办：在你的Drive中输入保存解压后的作业文件夹的路径，
# 例如 'cs231n/assignments/assignment3/'
FOLDERNAME = "cs231n/assignments/assignment3/"
assert FOLDERNAME is not None, "[!] 请输入文件夹名称。"

# 现在我们已经挂载了你的Drive，这确保了
# Colab虚拟机的Python解释器能够从其中加载
# Python文件
import sys
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME))

# 这会将COCO数据集下载到你的Drive中
# 如果它尚未存在的话
%cd /content/drive/My\ Drive/$FOLDERNAME/cs231n/datasets/  # 切换到数据集所在目录
!bash get_datasets.sh  # 执行shell脚本下载数据集
%cd /content/drive/My\ Drive/$FOLDERNAME  # 切换回作业主目录

## 使用GPU

前往 `Runtime > Change runtime type`（运行时 > 更改运行时类型），将 `Hardware accelerator`（硬件加速器）设置为 `GPU`（图形处理器）。这会重置Colab。**请重新运行顶部的单元格以再次挂载你的云端硬盘。**

## 自监督学习

### 什么是自监督学习？
现代机器学习需要大量带标签的数据。但很多时候，获取大量人工标注的数据既困难又昂贵。有没有一种方法能让机器自动学习一个模型，使其在没有标注数据集的情况下也能生成良好的视觉表征呢？答案是肯定的，这就是自监督学习！

自监督学习（SSL）能让模型利用给定数据集中的数据自动学习一个“良好的”表征空间，而无需依赖数据的标签。具体来说，如果我们的数据集是一堆图像，自监督学习可以让模型学习并生成图像的“良好的”表征向量。

自监督学习方法之所以大受欢迎，是因为通过这种方法训练出的模型在其他数据集上也能有出色表现——也就是说，在那些模型未经过训练的新数据集上同样适用！

### 什么是“良好的”表征？
“良好的”表征向量需要捕捉图像的重要特征，且这些特征能体现该图像与数据集中其他图像的关联。这意味着，数据集中代表语义相似实体的图像，其表征向量应该相似；而不同的图像，其表征向量应该不同。例如，两张苹果的图像应该有相似的表征向量，而一张苹果的图像和一张香蕉的图像则应该有不同的表征向量。

### 对比学习：SimCLR
最近，[SimCLR](https://arxiv.org/pdf/2002.05709.pdf) 提出了一种新的架构，它利用**对比学习**来学习良好的视觉表征。对比学习的目标是让相似图像具有相似的表征，让不同图像具有不同的表征。在本笔记本中我们会看到，这个简单的想法能让我们在不使用任何标签的情况下训练出一个效果惊人的模型。

具体来说，对于数据集中的每张图像，SimCLR会生成该图像的两个不同增强视图，称为**正样本对**。然后，模型会被促使为这对图像生成相似的表征向量。下图（来自论文中的图2）展示了该架构。

In [ ]:
# 运行此单元格以查看SimCLR架构图
from IPython.display import Image  # 从IPython.display导入Image类，用于显示图像
Image('images/simclr_fig2.png', width=500)  # 显示images目录下的simclr_fig2.png图片，宽度设为500像素

对于图像 **x** ，SimCLR使用两种不同的数据增强方式 **t** 和 **t'** 生成正样本对图像 **$\tilde{x}_i$** 和 **$\tilde{x}_j$**。$f$是一个基础编码器网络，用于从增强后的样本中提取表征向量，分别得到 **$h_i$** 和 **$h_j$** 。最后，一个小型神经网络投影头$g$将这些表征向量映射到应用对比损失的空间中。对比损失的目标是最大化最终向量 **$z_i = g(h_i)$**  和 **$z_j = g(h_j)$** 之间的一致性。稍后我们会更详细地讨论对比损失，你也将动手实现它。

训练完成后，我们会舍弃投影头$g$，只使用编码器$f$及其输出的表征$h$来执行下游任务，例如分类。你将有机会在训练好的SimCLR模型之上微调一个层来完成分类任务，并将其性能与基线模型（未使用自监督学习的模型）进行比较。

## 预训练权重
为了方便起见，我们为你提供了SimCLR模型的预训练权重（在CIFAR-10数据集上训练了约18小时）。运行以下单元格下载预训练模型权重，以备后续使用。（这大约需要1分钟）

In [ ]:
# %%bash

# 定义预训练权重保存目录
#DIR=pretrained_model/

# 如果目录不存在，则创建
#if [ ! -d "$DIR" ]; then
#    mkdir "$DIR"
#fi

# 预训练权重文件下载地址（注意：该链接曾解析失败，请确认可用性）
#URL=http://downloads.cs.stanford.edu/downloads/cs231n/pretrained_simclr_model.pth

# 定义权重文件最终保存路径
#FILE=pretrained_model/pretrained_simclr_model.pth

# 如果文件尚未存在，则开始下载
#if [ ! -f "$FILE" ]; then
#    echo "Downloading weights..."  # 正在下载权重...
#    wget "$URL" -O "$FILE"
#fi

In [ ]:
# 环境设置单元格
%pip install thop  # 安装thop库（用于计算模型的参数量和 FLOPs）
import torch  # 导入PyTorch库
import os  # 导入os库（用于文件和目录操作）
import importlib  # 导入importlib库（用于动态导入模块）
import pandas as pd  # 导入pandas库（用于数据处理和分析）
import numpy as np  # 导入numpy库（用于数值计算）
import torch.optim as optim  # 导入PyTorch的优化器模块
import torch.nn as nn  # 导入PyTorch的神经网络模块
import random  # 导入random库（用于随机数生成）
from thop import profile, clever_format  # 从thop库导入用于计算模型复杂度的函数
from torch.utils.data import DataLoader  # 导入DataLoader（用于数据加载）
from torchvision.datasets import CIFAR10  # 从torchvision导入CIFAR10数据集
import matplotlib.pyplot as plt  # 导入matplotlib用于绘图
# 设置matplotlib在 notebook 中 inline 显示
%matplotlib inline  

%load_ext autoreload  # 加载autoreload扩展（用于自动重新加载模块）
%autoreload 2  # 设置autoreload模式为2，即自动重新加载所有导入的模块

# 确定运行设备（优先使用GPU，若无则使用CPU）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 数据增强

我们的第一步是执行数据增强。在 `cs231n/simclr/data_utils.py` 中实现 `compute_train_transform()` 函数，以应用以下随机变换：

1. 随机调整大小并裁剪为 32x32
2. 以 0.5 的概率水平翻转图像
3. 以 0.8 的概率应用颜色抖动（参见 `compute_train_transform()` 中的定义）
4. 以 0.2 的概率将图像转换为灰度图

现在请完成 `cs231n/simclr/data_utils.py` 中的 `compute_train_transform()` 和 `CIFAR10Pair.__getitem__()` 函数，以应用数据增强变换并生成 **$\tilde{x}_i$** 和 **$\tilde{x}_j$**。


测试以确保你的数据增强代码是正确的：

In [ ]:
# 从cs231n.simclr.data_utils模块导入所有内容（包含数据增强相关函数和类）
from cs231n.simclr.data_utils import *
# 从cs231n.simclr.contrastive_loss模块导入所有内容（包含对比损失相关函数）
from cs231n.simclr.contrastive_loss import *

# 加载simclr_sanity_check.key文件中的验证数据（用于检查代码正确性）
answers = torch.load('simclr_sanity_check.key')

In [ ]:
from PIL import Image  # 导入PIL库的Image模块，用于图像处理
import torchvision  # 导入torchvision库，包含计算机视觉相关工具
from torchvision.datasets import CIFAR10  # 从torchvision.datasets导入CIFAR10数据集

def test_data_augmentation(correct_output=None):
    # 使用指定种子创建训练数据增强变换
    train_transform = compute_train_transform(seed=2147483647)
    # 加载CIFAR10训练集，应用上述数据增强变换，若本地无数据则下载
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
    # 创建数据加载器，批量大小为2，不打乱数据，使用2个工作进程
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=2, shuffle=False, num_workers=2)
    # 获取数据加载器的迭代器
    dataiter = iter(trainloader)
    # 从迭代器中获取一批图像和标签
    images, labels = next(dataiter)
    # 将图像拼接成网格以便展示
    img = torchvision.utils.make_grid(images)
    img = img / 2 + 0.5     # 反归一化（因为通常图像会被归一化到[-1,1]，这里恢复到[0,1]）
    npimg = img.numpy()  # 将张量转换为NumPy数组
    # 调整通道顺序并显示图像（PyTorch默认通道在前，Matplotlib需要通道在后）
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()  # 显示图像
    output = images  # 将增强后的图像作为输出

    # 计算并打印数据增强结果与正确输出之间的最大误差
    print("数据增强的最大误差: %g"%rel_error( output.numpy(), correct_output.numpy()))

# 该误差应小于1e-07
test_data_augmentation(answers['data_augmentation'])  # 使用答案中的数据增强结果进行测试

# 基础编码器和投影头

接下来的步骤是将基础编码器和投影头应用于增强样本 **$\tilde{x}_i$** 和 **$\tilde{x}_j$**。

基础编码器$f$用于提取增强样本的表征向量。SimCLR论文发现，使用更深、更宽的模型能提升性能，因此选择了[ResNet](https://arxiv.org/pdf/1512.03385.pdf)作为基础编码器。基础编码器的输出即为表征向量 **$h_i = f(\tilde{x}_i$)** 和 **$h_j = f(\tilde{x}_j$)** 。

投影头$g$是一个小型神经网络，它将表征向量 **$h_i$** 和 **$h_j$** 映射到应用对比损失的空间中。论文发现，使用非线性投影头能改善其前一层的表征质量。具体来说，他们使用了一个带一个隐藏层的MLP作为投影头$g$。随后，基于输出 **$z_i = g(h_i$)** 和 **$z_j = g(h_j$)** 计算对比损失。

我们在`cs231n/simclr/model.py`中提供了这两部分的实现。请浏览该文件，确保你理解其中的实现逻辑。

# SimCLR：对比损失

一个包含$N$个训练图像的小批量会生成总共$2N$个数据增强样本。对于增强样本的每个正样本对$(i, j)$，对比损失函数旨在最大化向量$z_i$和$z_j$的一致性。具体来说，该损失是归一化的温度缩放交叉熵损失，其目的是相对于批次中所有其他增强样本，最大化$z_i$和$z_j$的一致性：

$$
l \; (i, j) = -\log \frac{\exp (\;\text{sim}(z_i, z_j)\; / \;\tau) }{\sum_{k=1}^{2N} \mathbb{1}_{k \neq i} \exp (\;\text{sim} (z_i, z_k) \;/ \;\tau) }
$$

其中$\mathbb{1} \in \{0, 1\}$是一个指示函数，当$k \neq i$时输出1，否则输出0。$\tau$是一个温度参数，用于决定指数增长的速度。

sim $(z_i, z_j) = \frac{z_i \cdot z_j}{|| z_i || || z_j ||}$ 是向量$z_i$和$z_j$之间的（归一化）点积。$z_i$和$z_j$的相似度越高，点积就越大，分子也就越大。分母通过对$z_i$与批次中所有其他增强样本$k$的点积求和来进行归一化。归一化后的值范围为$(0, 1)$，其中接近1的高分对应着正样本对$(i, j)$具有高相似度，而$i$与批次中其他增强样本$k$的相似度较低。然后，负对数将范围$(0, 1)$映射到损失值$(\inf, 0)$。

总损失是通过计算批次中所有正样本对$(i, j)$的损失得到的。设$z = [z_1, z_2, ..., z_{2N}]$包含批次中所有的增强样本，其中$z_1...z_N$是左分支的输出，$z_{N+1}...z_{2N}$是右分支的输出。因此，正样本对为$(z_k, z_{k + N})$，其中$\forall k \in [1, N]$。

那么，总损失$L$为：

$$
L = \frac{1}{2N} \sum_{k=1}^N [ \; l(k, \;k+N) + l(k+N, \;k)\;]
$$

**注意：** 这个公式与论文中的公式略有不同。我们重新调整了批次中正样本对的顺序，因此索引有所不同。这种调整使得以向量化形式实现代码更加容易。

我们将逐步介绍如何以向量化形式实现这个损失函数。请在 `cs231n/simclr/contrastive_loss.py` 中实现 `sim` 函数和 `simclr_loss_naive` 函数。通过运行下面的完整性检查来测试你的代码。

In [ ]:
# 从cs231n.simclr.contrastive_loss模块导入所有内容（包含对比损失相关函数）
from cs231n.simclr.contrastive_loss import *
# 加载simclr_sanity_check.key文件中的验证数据（用于验证对比损失等代码的正确性）
answers = torch.load('simclr_sanity_check.key')

In [ ]:
def test_sim(left_vec, right_vec, correct_output):
    # 计算left_vec和right_vec的相似度，转换为NumPy数组（移至CPU进行处理）
    output = sim(left_vec, right_vec).cpu().numpy()
    # 打印相似度计算结果与正确输出之间的最大误差
    print("sim函数的最大误差: %g"%rel_error(correct_output.numpy(), output))

# 该误差应小于1e-07
# 测试第一组向量的相似度计算
test_sim(answers['left'][0], answers['right'][0], answers['sim'][0])
# 测试第二组向量的相似度计算
test_sim(answers['left'][1], answers['right'][1], answers['sim'][1])

In [ ]:
def test_loss_naive(left, right, tau, correct_output):
    # 计算朴素版本的SimCLR损失，将结果转换为Python标量
    naive_loss = simclr_loss_naive(left, right, tau).item()
    # 打印朴素损失函数计算结果与正确输出之间的最大误差
    print("simclr_loss_naive函数的最大误差: %g"%rel_error(correct_output, naive_loss))

# 该误差应小于1e-07
# 使用温度参数5.0测试朴素损失函数
test_loss_naive(answers['left'], answers['right'], 5.0, answers['loss']['5.0'])
# 使用温度参数1.0测试朴素损失函数
test_loss_naive(answers['left'], answers['right'], 1.0, answers['loss']['1.0'])

现在通过在 `cs231n/simclr/contrastive_loss.py` 中实现 `sim_positive_pairs`、`compute_sim_matrix` 和 `simclr_loss_vectorized` 函数，来完成向量化版本的实现。运行下面的完整性检查来测试你的代码。

In [ ]:
def test_sim_positive_pairs(left, right, correct_output):
    # 计算正样本对的相似度，将结果转换为NumPy数组（移至CPU处理）
    sim_pair = sim_positive_pairs(left, right).cpu().numpy()
    # 打印正样本对相似度计算结果与正确输出之间的最大误差
    print("sim_positive_pairs函数的最大误差: %g"%rel_error(correct_output.numpy(), sim_pair))

# 该误差应小于1e-07
# 测试正样本对相似度计算函数
test_sim_positive_pairs(answers['left'], answers['right'], answers['sim'])

In [ ]:
def test_sim_matrix(left, right, correct_output):
    # 将左分支和右分支的输出拼接在一起，形成包含所有样本的矩阵
    out = torch.cat([left, right], dim=0)
    # 计算相似度矩阵，并将结果移至CPU
    sim_matrix = compute_sim_matrix(out).cpu()
    # 断言相似度矩阵与正确输出是否接近，若不接近则输出错误信息
    assert torch.isclose(sim_matrix, correct_output).all(), "正确值: {}. 实际得到: {}".format(correct_output, sim_matrix)
    # 若断言通过，则打印测试通过信息
    print("测试通过!")

# 测试相似度矩阵计算函数
test_sim_matrix(answers['left'], answers['right'], answers['sim_matrix'])

In [ ]:
def test_loss_vectorized(left, right, tau, correct_output):
    # 计算向量化版本的SimCLR损失，指定设备与left张量相同，并转换为Python标量
    vec_loss = simclr_loss_vectorized(left, right, tau, device=left.device).item()
    # 打印向量化损失函数计算结果与正确输出之间的最大误差
    print("向量化损失函数的最大误差: %g"%rel_error(correct_output, vec_loss))

# 该误差应小于1e-07
# 使用温度参数5.0测试向量化损失函数
test_loss_vectorized(answers['left'], answers['right'], 5.0, answers['loss']['5.0'])
# 使用温度参数1.0测试向量化损失函数
test_loss_vectorized(answers['left'], answers['right'], 1.0, answers['loss']['1.0'])

# 实现训练函数
请完成 `cs231n/simclr/utils.py` 中的 `train()` 函数，以获取模型的输出并使用 `simclr_loss_vectorized` 计算损失。（请查看 `cs231n/simclr/model.py` 中的 `Model` 类，以理解模型流程和返回值）

In [ ]:
# 从cs231n.simclr.data_utils模块导入所有内容（包含数据增强相关的函数和类）
from cs231n.simclr.data_utils import *
# 从cs231n.simclr.model模块导入所有内容（包含模型结构相关的类，如基础编码器、投影头等）
from cs231n.simclr.model import *
# 从cs231n.simclr.utils模块导入所有内容（包含训练相关的工具函数，如训练函数等）
from cs231n.simclr.utils import *

### 训练SimCLR模型

运行以下单元格以加载预训练权重并继续训练一段时间。这部分将花费约10分钟，训练结果将输出到`pretrained_model/trained_simclr_model.pth`。

**注意：** 不必担心诸如“_[WARN] 找不到……的规则_”之类的日志。这些日志与笔记本中使用的另一个模块有关。你可以通过我们提供的提示和注释来验证代码更改的完整性。

In [ ]:
# 请勿修改此单元格
feature_dim = 128  # 特征维度
temperature = 0.5  # 温度参数
k = 200  # 近邻数量
batch_size = 64  # 批次大小
epochs = 1  # 训练轮数
temperature = 0.5  # 温度参数（重复定义，可能用于强调或后续覆盖）
percentage = 0.5  # 使用训练数据的比例
pretrained_path = './pretrained_model/pretrained_simclr_model.pth'  # 预训练模型路径

# 准备数据
train_transform = compute_train_transform()  # 获取训练数据增强变换
# 加载CIFAR10Pair训练集，应用训练变换，若本地无数据则下载
train_data = CIFAR10Pair(root='data', train=True, transform=train_transform, download=True)
# 取训练数据的一部分（按percentage比例）作为子集
train_data = torch.utils.data.Subset(train_data, list(np.arange(int(len(train_data)*percentage))))
# 创建训练数据加载器
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True, drop_last=True)
test_transform = compute_test_transform()  # 获取测试数据变换
# 加载用于记忆库的CIFAR10Pair数据集（训练集），应用测试变换
memory_data = CIFAR10Pair(root='data', train=True, transform=test_transform, download=True)
# 创建记忆库数据加载器
memory_loader = DataLoader(memory_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)
# 加载CIFAR10Pair测试集，应用测试变换
test_data = CIFAR10Pair(root='data', train=False, transform=test_transform, download=True)
# 创建测试数据加载器
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

# 设置模型和优化器配置
model = Model(feature_dim)  # 初始化模型，指定特征维度
# 加载预训练模型权重（映射到CPU），非严格模式（允许部分权重不匹配）
model.load_state_dict(torch.load(pretrained_path, map_location='cpu'), strict=False)
model = model.to(device)  # 将模型移至指定设备（如GPU）
# 计算模型的FLOPs（浮点运算次数）和参数数量
flops, params = profile(model, inputs=(torch.randn(1, 3, 32, 32).to(device),))
flops, params = clever_format([flops, params])  # 格式化FLOPs和参数数量的显示
print('# 模型参数: {} 浮点运算数: {}'.format(params, flops))
# 初始化Adam优化器
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-6)
c = len(memory_data.classes)  # 获取类别数量

# 训练循环
results = {'train_loss': [], 'test_acc@1': [], 'test_acc@5': []}  # 用于存储训练结果的字典

if not os.path.exists('results'):  # 若results目录不存在则创建
    os.mkdir('results')
best_acc = 0.0  # 记录最佳准确率
for epoch in range(1, epochs + 1):  # 遍历每个训练轮次
    # 训练模型，返回训练损失
    train_loss = train(model, train_loader, optimizer, epoch, epochs, batch_size=batch_size, temperature=temperature, device=device)
    results['train_loss'].append(train_loss)  # 记录训练损失
    # 测试模型，返回top-1和top-5准确率
    test_acc_1, test_acc_5 = test(model, memory_loader, test_loader, epoch, epochs, c, k=k, temperature=temperature, device=device)
    results['test_acc@1'].append(test_acc_1)  # 记录top-1准确率
    results['test_acc@5'].append(test_acc_5)  # 记录top-5准确率

    # 保存统计结果
    if test_acc_1 > best_acc:  # 若当前top-1准确率高于最佳准确率
        best_acc = test_acc_1  # 更新最佳准确率
        # 保存模型状态字典
        torch.save(model.state_dict(), './pretrained_model/trained_simclr_model.pth')

# 微调一个线性层用于分类微调！

现在是检验表征向量的时候了！

我们从SimCLR模型中移除投影头，添加一个线性层，针对简单的分类任务进行微调。线性层之前的所有层都被冻结，只有最后的线性层中的权重会被训练。我们将SimCLR+微调模型的性能与基线模型进行比较——基线模型没有事先进行自监督学习，模型中的所有权重都会被训练。你将亲身体验到自监督学习的力量，以及学到的表征向量如何提升下游任务的性能。


## 基线模型：不使用自监督学习
首先，我们来看一下基线模型。我们会从SimCLR模型中移除投影头，添加一个线性层，针对简单的分类任务进行微调。该模型没有事先进行自监督学习，模型中的所有权重都会被训练。运行以下单元格。

**注意：** 如果看到模型性能较低但尚合理，不必担心。

In [ ]:
class Classifier(nn.Module):
    def __init__(self, num_class):
        super(Classifier, self).__init__()  # 调用父类nn.Module的初始化方法

        # 编码器（使用SimCLR模型中的基础编码器部分）
        self.f = Model().f

        # 分类器（线性层）
        # 将2048维的特征映射到num_class维（对应分类任务的类别数），包含偏置项
        self.fc = nn.Linear(2048, num_class, bias=True)

    def forward(self, x):
        x = self.f(x)  # 通过编码器提取特征
        feature = torch.flatten(x, start_dim=1)  # 将特征展平（从第1维开始，保留批次维度）
        out = self.fc(feature)  # 通过线性层得到分类输出
        return out

In [ ]:
# 请勿修改此单元格
feature_dim = 128  # 特征维度
temperature = 0.5  # 温度参数
k = 200  # 近邻数量
batch_size = 128  # 批次大小
epochs = 10  # 训练轮数
percentage = 0.1  # 使用训练数据的比例

# 准备数据
train_transform = compute_train_transform()  # 获取训练数据增强变换
# 加载CIFAR10训练集，应用训练变换，若本地无数据则下载
train_data = CIFAR10(root='data', train=True, transform=train_transform, download=True)
# 取训练数据的一部分（按percentage比例）作为子集
trainset = torch.utils.data.Subset(train_data, list(np.arange(int(len(train_data)*percentage))))
# 创建训练数据加载器
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
test_transform = compute_test_transform()  # 获取测试数据变换
# 加载CIFAR10测试集，应用测试变换
test_data = CIFAR10(root='data', train=False, transform=test_transform, download=True)
# 创建测试数据加载器
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

# 初始化分类器模型并移至指定设备（如cuda）
model = Classifier(num_class=len(train_data.classes)).to(device)
# 冻结编码器部分的参数（不参与训练）
for param in model.f.parameters():
    param.requires_grad = False

# 计算模型的FLOPs（浮点运算次数）和参数数量
flops, params = profile(model, inputs=(torch.randn(1, 3, 32, 32).to(device),))
flops, params = clever_format([flops, params])  # 格式化FLOPs和参数数量的显示
print('# 模型参数: {} 浮点运算数: {}'.format(params, flops))
# 初始化Adam优化器（仅优化分类器的线性层参数）
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3, weight_decay=1e-6)
# 用于存储无预训练模型的训练结果
no_pretrain_results = {'train_loss': [], 'train_acc@1': [], 'train_acc@5': [],
           'test_loss': [], 'test_acc@1': [], 'test_acc@5': []}

best_acc = 0.0  # 记录最佳准确率
for epoch in range(1, epochs + 1):  # 遍历每个训练轮次
    # 训练模型，返回训练损失、top-1和top-5准确率
    train_loss, train_acc_1, train_acc_5 = train_val(model, train_loader, optimizer, epoch, epochs, device='cuda')
    # 记录训练结果
    no_pretrain_results['train_loss'].append(train_loss)
    no_pretrain_results['train_acc@1'].append(train_acc_1)
    no_pretrain_results['train_acc@5'].append(train_acc_5)
    # 在测试集上验证，返回测试损失、top-1和top-5准确率（不传入优化器，即仅验证）
    test_loss, test_acc_1, test_acc_5 = train_val(model, test_loader, None, epoch, epochs)
    # 记录测试结果
    no_pretrain_results['test_loss'].append(test_loss)
    no_pretrain_results['test_acc@1'].append(test_acc_1)
    no_pretrain_results['test_acc@5'].append(test_acc_5)
    # 更新最佳准确率
    if test_acc_1 > best_acc:
        best_acc = test_acc_1

# 打印最佳测试准确率
print('无自监督学习的最佳top-1准确率: ', best_acc)

## 采用自监督学习

让我们看看通过自监督学习能获得多大的性能提升。在这里，我们使用你编写的SimCLR损失函数对SimCLR模型进行预训练，然后从该模型中移除投影头，再用一个线性层针对简单的分类任务进行微调。

In [ ]:
# 请勿修改此单元格
feature_dim = 128  # 特征维度
temperature = 0.5  # 温度参数
k = 200  # 近邻数量
batch_size = 128  # 批次大小
epochs = 10  # 训练轮数
percentage = 0.1  # 使用训练数据的比例
pretrained_path = './pretrained_model/trained_simclr_model.pth'  # 预训练模型路径

# 准备数据
train_transform = compute_train_transform()  # 获取训练数据增强变换
# 加载CIFAR10训练集，应用训练变换，若本地无数据则下载
train_data = CIFAR10(root='data', train=True, transform=train_transform, download=True)
# 取训练数据的一部分（按percentage比例）作为子集
trainset = torch.utils.data.Subset(train_data, list(np.arange(int(len(train_data)*percentage))))
# 创建训练数据加载器
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16, pin_memory=True)
test_transform = compute_test_transform()  # 获取测试数据变换
# 加载CIFAR10测试集，应用测试变换
test_data = CIFAR10(root='data', train=False, transform=test_transform, download=True)
# 创建测试数据加载器
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=16, pin_memory=True)

# 初始化分类器模型（指定类别数）
model = Classifier(num_class=len(train_data.classes))
# 加载预训练的SimCLR模型权重（映射到CPU），非严格模式（允许部分权重不匹配）
model.load_state_dict(torch.load(pretrained_path, map_location='cpu'), strict=False)
model = model.to(device)  # 将模型移至指定设备（如GPU）
# 冻结编码器部分的参数（不参与训练）
for param in model.f.parameters():
    param.requires_grad = False

# 计算模型的FLOPs（浮点运算次数）和参数数量
flops, params = profile(model, inputs=(torch.randn(1, 3, 32, 32).to(device),))
flops, params = clever_format([flops, params])  # 格式化FLOPs和参数数量的显示
print('# 模型参数: {} 浮点运算数: {}'.format(params, flops))
# 初始化Adam优化器（仅优化分类器的线性层参数）
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3, weight_decay=1e-6)
# 用于存储有预训练模型的训练结果
pretrain_results = {'train_loss': [], 'train_acc@1': [], 'train_acc@5': [],
           'test_loss': [], 'test_acc@1': [], 'test_acc@5': []}

best_acc = 0.0  # 记录最佳准确率
for epoch in range(1, epochs + 1):  # 遍历每个训练轮次
    # 训练模型，返回训练损失、top-1和top-5准确率
    train_loss, train_acc_1, train_acc_5 = train_val(model, train_loader, optimizer, epoch, epochs)
    # 记录训练结果
    pretrain_results['train_loss'].append(train_loss)
    pretrain_results['train_acc@1'].append(train_acc_1)
    pretrain_results['train_acc@5'].append(train_acc_5)
    # 在测试集上验证，返回测试损失、top-1和top-5准确率（不传入优化器，即仅验证）
    test_loss, test_acc_1, test_acc_5 = train_val(model, test_loader, None, epoch, epochs)
    # 记录测试结果
    pretrain_results['test_loss'].append(test_loss)
    pretrain_results['test_acc@1'].append(test_acc_1)
    pretrain_results['test_acc@5'].append(test_acc_5)
    # 更新最佳准确率
    if test_acc_1 > best_acc:
        best_acc = test_acc_1

# 打印最佳测试准确率。你应该会看到最佳top-1准确率≥70%。
print('采用自监督学习的最佳top-1准确率: ', best_acc)

### 绘制对比图表

绘制基线模型（无预训练）和经自监督学习预训练的同一模型之间的测试准确率对比图。

In [ ]:
# 绘制无预训练模型的测试top-1准确率曲线，标签为“Without Pretrain”
plt.plot(no_pretrain_results['test_acc@1'], label="无预训练")
# 绘制有预训练模型的测试top-1准确率曲线，标签为“With Pretrain”
plt.plot(pretrain_results['test_acc@1'], label="有预训练")
# 设置x轴标签为“Epochs”（轮次）
plt.xlabel('轮次')
# 设置y轴标签为“Accuracy”（准确率）
plt.ylabel('准确率')
# 设置图表标题为“Test Top-1 Accuracy”（测试集top-1准确率）
plt.title('测试集top-1准确率')
# 显示图例
plt.legend()
# 展示图表
plt.show()